In [1]:
!mkdir datasets/3rdfloor/sparse_txt
!colmap model_converter --input_path datasets/3rdfloor/sparse/0 \
                        --output_path datasets/3rdfloor/sparse_txt --output_type TXT

In [2]:
base_path = 'datasets/3rdfloor'

In [3]:
from scipy.spatial.transform import Rotation as R
import numpy as np
import os
from tqdm import trange

In [4]:
in_path = os.path.join(base_path, 'sparse_txt')
out_path = os.path.join(base_path, 'poses')
os.mkdir(out_path)

poses = open(os.path.join(in_path, 'images.txt'), 'r')
camera = open(os.path.join(in_path, 'cameras.txt'), 'r')

lines_cam = camera.readlines()
intrinsics = lines_cam[3].split()
fp = intrinsics[4]
cx, cy = intrinsics[5], intrinsics[6]
intr = np.eye(3)
intr[0, 0] = fp
intr[1, 1] = fp
intr[0, 2] = cx
intr[1, 2] = cy
np.savetxt(os.path.join(base_path, 'intrinsics.txt'), intr)

lines_poses = poses.readlines()
for i in trange(4, len(lines_poses)):
    if i % 2 == 0:
        x = lines_poses[i].split()
        QW = x[1]
        QX = x[2]
        QY = x[3]
        QZ = x[4]
        TX = x[5]
        TY = x[6]
        TZ = x[7]
        fname = x[9].replace('jpg', 'txt')
        r = R.from_quat([QX, QY, QZ, QW]).as_matrix()
        t = np.array([TX, TY, TZ]).astype('float')
        cam_to_world = np.eye(4)
        cam_to_world[:3,:3] = r.T
        cam_to_world[:3,3] = np.matmul(-r.T, t)
        np.savetxt(os.path.join(out_path, fname), cam_to_world)

100%|██████████| 1650/1650 [00:00<00:00, 4600.89it/s]
